# Example of MagSvc class

This demonstrates direct access to the bound c++ functions. It is an advanced use case.

The notebook can be downloaded [here](https://github.com/lephare-photoz/lephare/blob/main/docs/notebooks/Example_of_usage_of_magSvc.ipynb).

In [ ]:
import lephare as lp
import numpy as np
from matplotlib import pylab as plt
from scipy import integrate as sciint
import glob, time

%matplotlib inline

In [ ]:
keymap = lp.all_types_to_keymap(lp.default_cosmos_config)
# Get the auxiliary files required.
lp.data_retrieval.get_auxiliary_data(
    keymap=keymap, additional_files=["sed/STAR/BD_NEW/lte012.0-4.0-0.0a+0.0.BT-Settl.spec.txt"]
)

In [ ]:
# we can write the config to a file to keep a record
config_file = "./config_file.para"
lp.write_para_config(keymap, config_file)
allFlt = lp.FilterSvc.from_config(config_file)

In [ ]:
sed = lp.StarSED("test", 0)
sed.read(f"{lp.LEPHAREDIR}/sed/STAR/BD_NEW/lte012.0-4.0-0.0a+0.0.BT-Settl.spec.txt")

In [ ]:
opavec = lp.GalMag.read_opa()

In [ ]:
# We need the full previous stages to get the mags
filterLib = lp.Filter(config_file=config_file)
filterLib.run()
sedlib = lp.Sedtolib(config_keymap=keymap)
sedlib.run(typ="STAR", star_sed="$LEPHAREDIR/sed/STAR/STAR_MOD_ALL.list")
sedlib.run(typ="QSO", qso_sed="$LEPHAREDIR/sed/QSO/SALVATO09/AGN_MOD.list", gal_lib="LIB_QSO")
sedlib.run(typ="GAL", gal_sed="$LEPHAREDIR/sed/GAL/COSMOS_SED/COSMOS_MOD.list", gal_lib="LIB_GAL")
maglib = lp.MagGal(config_keymap=keymap)
maglib.run(typ="STAR", lib_ascii="YES", star_lib_out="ALLSTAR_COSMOS")
maglib.run(
    typ="QSO",
    lib_ascii="YES",
    mod_extinc="0,1000",
    eb_v="0.,0.1,0.2,0.3",
    extinc_law="SB_calzetti.dat",
    qso_lib_in="LIB_QSO",
)
maglib.run(
    typ="GAL",
    lib_ascii="YES",
    gal_lib_in="LIB_GAL",
    gal_lib_out="VISTA_COSMOS_FREE",
    mod_extinc="18,26,26,33,26,33,26,33",
    extinc_law="SMC_prevot.dat,SB_calzetti.dat,SB_calzetti_bump1.dat,SB_calzetti_bump2.dat",
    em_lines="EMP_UV",
    em_dispersion="0.5,0.75,1.,1.5,2.",
)

In [ ]:
mag = lp.MagSvc.from_config("Star", config_file)

In [ ]:
newsed = mag.make_maglib(sed)

In [ ]:
newsed[0].mag